## ETL Project - World Happiness and Global Peace Index based on Alcohol Consumption 2016

### Resorces:
* [Kaggle](https://www.kaggle.com/)
    - gpi_2008-2016.csv
    - HappinessAlcoholConsumption.csv

# ==========================================

### schema.sql

```sql
-- Create Tables
CREATE TABLE alc_consumption_happiness (
    country VARCHAR (60) PRIMARY KEY,
	region VARCHAR (60),
    happiness_score2016 DECIMAL,
    beer_per_capita INT,
    spirit_per_capita INT,
    wine_per_capita INT
);

CREATE TABLE global_peace_index (
    country VARCHAR (60) PRIMARY KEY,
    gpi_score2016 DECIMAL
);

```

### query.sql

```sql
-- Query to check successful load
SELECT * FROM alc_consumption_happiness;

SELECT * FROM global_peace_index;

-- Join tables on 'country'
SELECT 
alc_consumption_happiness.country, 
alc_consumption_happiness.region, 
alc_consumption_happiness.happiness_score2016,
global_peace_index.gpi_score2016,
alc_consumption_happiness.beer_per_capita, 
alc_consumption_happiness.spirit_per_capita, 
alc_consumption_happiness.wine_per_capita
FROM alc_consumption_happiness
INNER JOIN global_peace_index
ON alc_consumption_happiness.country = global_peace_index.country;

-- Additional queries
SELECT country AS "happiest country", happiness_score2016 
FROM alc_consumption_happiness
ORDER BY happiness_score2016 DESC;

SELECT country AS "most peaceful country", gpi_score2016 
FROM global_peace_index
ORDER BY gpi_score2016;

SELECT 
country, 
beer_per_capita AS "beer consumption", 
happiness_score2016 AS "happiness score"
FROM alc_consumption_happiness  
ORDER BY beer_per_capita DESC
LIMIT 10;

SELECT 
country, 
spirit_per_capita AS "spirit consumption", 
happiness_score2016 AS "happiness score"
FROM alc_consumption_happiness  
ORDER BY spirit_per_capita DESC
LIMIT 10;

SELECT 
country, 
wine_per_capita AS "wine consumption", 
happiness_score2016 AS "happiness score"
FROM alc_consumption_happiness  
ORDER BY wine_per_capita DESC
LIMIT 10;
```

In [1]:
import pandas as pd
from sqlalchemy import create_engine
from config import password

### Extract CSVs into DataFrames

In [2]:
alcoholHappiness = "Resources/HappinessAlcoholConsumption.csv"
alcoholHappiness_df = pd.read_csv(alcoholHappiness)
alcoholHappiness_df.head()

,Country,Region,Hemisphere,HappinessScore,HDI,GDP_PerCapita,Beer_PerCapita,Spirit_PerCapita,Wine_PerCapita
0,Denmark,Western Europe,north,7.526,928,53.579,224,81,278
1,Switzerland,Western Europe,north,7.509,943,79.866,185,100,280
2,Iceland,Western Europe,north,7.501,933,60.530,233,61,78
3,Norway,Western Europe,north,7.498,951,70.890,169,71,129
4,Finland,Western Europe,north,7.413,918,43.433,263,133,97


In [3]:
globalPeaceIndex = "Resources/gpi_2008-2016.csv"
globalPeaceIndex_df = pd.read_csv(globalPeaceIndex)
globalPeaceIndex_df.head()

,country,score_2008,score_2009,score_2010,score_2011,score_2012,score_2013,score_2014,score_2015,score_2016
0,Iceland,1.107,1.203,1.212,1.148,1.113,1.162,1.189,1.148,1.192
1,Denmark,1.238,1.263,1.341,1.289,1.239,1.207,1.193,1.150,1.246
2,Austria,1.291,1.240,1.290,1.337,1.328,1.250,1.200,1.198,1.278
3,New Zealand,1.190,1.227,1.188,1.279,1.239,1.237,1.236,1.221,1.287
4,Portugal,1.385,1.426,1.366,1.453,1.470,1.467,1.425,1.344,1.356


### Transform alcoholHappiness DataFrame

In [4]:
# Create a filtered dataframe from specific columns
alcoholHappiness_df = alcoholHappiness_df[['Country',
                                           'Region',
                                           'HappinessScore', 
                                           'Beer_PerCapita', 
                                           'Spirit_PerCapita', 
                                           'Wine_PerCapita']].copy()
alcoholHappiness_df.head()

,Country,Region,HappinessScore,Beer_PerCapita,Spirit_PerCapita,Wine_PerCapita
0,Denmark,Western Europe,7.526,224,81,278
1,Switzerland,Western Europe,7.509,185,100,280
2,Iceland,Western Europe,7.501,233,61,78
3,Norway,Western Europe,7.498,169,71,129
4,Finland,Western Europe,7.413,263,133,97


In [5]:
# Rename the column headers
alcoholHappiness_df = alcoholHappiness_df.rename(columns={"Country": "country",
                                                          "Region": "region",
                                                          "HappinessScore": "happiness_score2016",
                                                          "Beer_PerCapita": "beer_per_capita", 
                                                          "Spirit_PerCapita": "spirit_per_capita",
                                                          "Wine_PerCapita": "wine_per_capita"})

# Clean the data by dropping duplicates and setting the index
alcoholHappiness_df.drop_duplicates("country", inplace=True)
alcoholHappiness_df.set_index("country", inplace=True)

alcoholHappiness_df.head()

,region,happiness_score2016,beer_per_capita,spirit_per_capita,wine_per_capita
country,,,,,
Denmark,Western Europe,7.526,224,81,278
Switzerland,Western Europe,7.509,185,100,280
Iceland,Western Europe,7.501,233,61,78
Norway,Western Europe,7.498,169,71,129
Finland,Western Europe,7.413,263,133,97


### Transform globalPeaceIndex DataFrame

In [6]:
globalPeaceIndex_df = globalPeaceIndex_df[['country', 'score_2016']].copy()
globalPeaceIndex_df.head()

,country,score_2016
0,Iceland,1.192
1,Denmark,1.246
2,Austria,1.278
3,New Zealand,1.287
4,Portugal,1.356


In [7]:
# Rename the column headers
globalPeaceIndex_df = globalPeaceIndex_df.rename(columns={"score_2016": "gpi_score2016"})

# Clean the data by dropping duplicates and setting the index
globalPeaceIndex_df.drop_duplicates("country", inplace=True)
globalPeaceIndex_df.set_index("country", inplace=True)

globalPeaceIndex_df.head()

,gpi_score2016
country,
Iceland,1.192
Denmark,1.246
Austria,1.278
New Zealand,1.287
Portugal,1.356


### Create database connection

In [8]:
# connection_string = "postgres:postgres@localhost:5432/customer_db"
connection_string = "postgres:"+password+"@localhost:5432/alcoholHappinessGPI"
engine = create_engine(f'postgresql://{connection_string}')

In [9]:
# Confirm tables
engine.table_names()

['alc_consumption_happiness', 'global_peace_index']

### Load DataFrames into database

In [10]:
alcoholHappiness_df.to_sql(name='alc_consumption_happiness', con=engine, if_exists='append', index=True)

In [11]:
globalPeaceIndex_df.to_sql(name='global_peace_index', con=engine, if_exists='append', index=True)

#### Table queries using sql

In [12]:
pd.read_sql_query('select * from alc_consumption_happiness', con=engine).head()

,country,region,happiness_score2016,beer_per_capita,spirit_per_capita,wine_per_capita
0,Denmark,Western Europe,7.526,224,81,278
1,Switzerland,Western Europe,7.509,185,100,280
2,Iceland,Western Europe,7.501,233,61,78
3,Norway,Western Europe,7.498,169,71,129
4,Finland,Western Europe,7.413,263,133,97


In [13]:
pd.read_sql_query('select * from global_peace_index', con=engine).head()

,country,gpi_score2016
0,Iceland,1.192
1,Denmark,1.246
2,Austria,1.278
3,New Zealand,1.287
4,Portugal,1.356
